In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [2]:
# calling biomarkers
bloodbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk1_20231117.csv"
bloodbiomk1 = pd.read_csv(bloodbiomk1_file_path)
bloodbiomk1 = pd.DataFrame(bloodbiomk1)

bloodbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_20231117.csv"
bloodbiomk2 = pd.read_csv(bloodbiomk2_file_path)
bloodbiomk2 = pd.DataFrame(bloodbiomk2)

csfbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk1_20231117.csv"
csfbiomk1 = pd.read_csv(csfbiomk1_file_path)
csfbiomk1 = pd.DataFrame(csfbiomk1)

csfbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk2_20231117.csv"
csfbiomk2 = pd.read_csv(csfbiomk2_file_path)
csfbiomk2 = pd.DataFrame(csfbiomk2)

csfbiomk3_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk3_20231117.csv"
csfbiomk3 = pd.read_csv(csfbiomk3_file_path)
csfbiomk3 = pd.DataFrame(csfbiomk3)


In [3]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'blood_biomk1_to_briefa'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)

# save_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\blood_biomk1_briefa" # Update this path to your desired folder


# Preparing Bloodbiomarkers 1

In [4]:
# Check for null values in each column
bloodbiomk1_nulls = bloodbiomk1.isnull().sum()

bloodbiomk1_nulls_df = pd.DataFrame({
    'Column': bloodbiomk1_nulls.index,
    'Number of Nulls': bloodbiomk1_nulls.values
})

bloodbiomk1_nulls_df_transposed = bloodbiomk1_nulls_df.T

# print("Null values in bloodbiomk1:")
# print(bloodbiomk1_nulls_df)
bloodbiomk1_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk1_nulls.csv", index=False)


In [5]:
bloodbiomk1 = bloodbiomk1.drop(columns=bloodbiomk1.loc[:,'subject_id':'visit'].columns)
bloodbiomk1 = bloodbiomk1.drop(columns=bloodbiomk1.loc[:,'exposurebin':'chiiyrs_pg'].columns)

bloodbiomk1 = bloodbiomk1.drop(columns='p_PDGFRbeta_FLAG_other')
bloodbiomk1 = bloodbiomk1.drop(columns=['p_Ab40_FLAG_below_ref_range',
                              'p_Ab42_FLAG_below_ref_range',
                              'p_GFAP_FLAG_below_ref_range', 
                              'p_NfL_FLAG_below_ref_range',
                              'dxcte_totaltau_plasma_flag_dv'])

bloodbiomk1.head()

,checkin_bin,p_Ab40,p_Ab42,p_GFAP,p_NfL,p_PDGFRbeta,p_pT181,p_pT231,p_ttau
0,2,126.5,8.535,73.80,11.95,5628.71,8.12,7.58,1.25
1,2,84.2,7.460,42.05,6.44,10123.09,6.04,7.53,1.20
2,2,124.0,7.285,34.35,8.22,10045.39,9.33,9.34,1.57
3,1,110.5,5.920,48.45,13.00,9563.19,6.63,5.87,0.73
4,3,126.0,9.150,46.75,11.20,12826.15,8.31,7.99,1.54


In [6]:
bloodbiomk1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   checkin_bin  236 non-null    int64  
 1   p_Ab40       232 non-null    float64
 2   p_Ab42       231 non-null    float64
 3   p_GFAP       228 non-null    float64
 4   p_NfL        232 non-null    float64
 5   p_PDGFRbeta  231 non-null    float64
 6   p_pT181      232 non-null    float64
 7   p_pT231      232 non-null    float64
 8   p_ttau       226 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 16.7 KB


In [7]:
# Calculate the mean for each column, ignoring NaNs
for column in bloodbiomk1.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = bloodbiomk1.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            bloodbiomk1.loc[(bloodbiomk1['checkin_bin'] == label) & (bloodbiomk1[column].isnull()), column] = averages[label]

# print(bloodbiomk1.head())

In [8]:
bloodbiomk1 = bloodbiomk1.drop(columns='checkin_bin')


# Preparing BloodBiomarkers 2

In [9]:
# Check for null values in each column
bloodbiomk2_nulls = bloodbiomk2.isnull().sum()

bloodbiomk2_nulls_df = pd.DataFrame({
    'Column': bloodbiomk2_nulls.index,
    'Number of Nulls': bloodbiomk2_nulls.values
})

bloodbiomk2_nulls_df_transposed = bloodbiomk2_nulls_df.T
bloodbiomk2_nulls_df_sorted = bloodbiomk2_nulls_df.sort_values(by='Number of Nulls', ascending=False)

# print("Null values in bloodbiomk2 (sorted):")
# print(bloodbiomk2_nulls_df_sorted)

# print("Null values in bloodbiomk2:")
# print(bloodbiomk2_nulls_df)
# bloodbiomk2_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_nulls.csv", index=False)
bloodbiomk2_nulls_df_sorted.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_nulls_sorted.csv", index=False)


In [10]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in bloodbiomk2.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = bloodbiomk2.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            bloodbiomk2.loc[(bloodbiomk2['checkin_bin'] == label) & (bloodbiomk2[column].isnull()), column] = averages[label]

# print(bloodbiomk2.head())

bloodbiomk2.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_nullreplaced.csv")


In [11]:
bloodbiomk2 = bloodbiomk2.drop(columns=bloodbiomk2.loc[:,'subject_id':'chiiyrs_pg'].columns)
# bloodbiomk2 = bloodbiomk2.drop(columns='p_PDGFRbeta_FLAG_other') # This is not for this dataset unless you find a column where there are too many nulls

bloodbiomk2.head()

,Qalb,s_alb,p_IL_17A_F_cv,p_IL_17A_F,p_IL_17B_cv,p_IL_17B,p_IL_17C_cv,p_IL_17C,p_IL_17D_cv,p_IL_17D,...,p_IL_7_cv,p_IL_7,p_TNF_beta_cv,p_TNF_beta,p_VEGF_A_cv,p_VEGF_A,p_asyn_cv,p_asyn,p_Hb,dxcte_asyn_p_flag_dv
0,7.852013,3.912500,5.43,2.190,1.63,1.18,7.39,1.59,14.40,28.40,...,3.060,3.06,6.040,0.1210,0.943,16.6,0.51,178000.0,37538.80,0.0
1,6.994813,3.556088,6.84,3.460,4.52,2.30,3.30,2.23,8.79,34.30,...,2.440,12.30,0.331,0.2850,2.640,55.8,12.60,170000.0,26355.70,0.0
2,6.994813,3.556088,2.49,0.126,2.63,2.30,7.44,17.50,27.00,9.01,...,3.960,1.87,15.400,0.8180,5.680,25.3,0.14,79500.0,148871.00,0.0
3,7.867197,3.414635,1.05,8.280,5.71,2.95,9.27,1.94,6.47,35.30,...,5.940,9.81,1.860,0.0472,0.192,62.5,5.33,117000.0,20893.80,0.0
4,7.429948,3.378848,18.90,1.180,2.58,1.16,3.81,6.84,26.20,26.10,...,0.428,8.12,0.306,0.1560,5.370,99.2,4.54,101000.0,8102.65,0.0


# Preparing CSF Biomrarkers 1

In [12]:
# Check for null values in each column
csfbiomk1_nulls = csfbiomk1.isnull().sum()

csfbiomk1_nulls_df = pd.DataFrame({
    'Column': csfbiomk1_nulls.index,
    'Number of Nulls': csfbiomk1_nulls.values
})

csfbiomk1_nulls_df_transposed = csfbiomk1_nulls_df.T

# print("Null values in csfbiomk1:")
# print(csfbiomk1_nulls_df)
csfbiomk1_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk1_nulls.csv", index=False)


In [13]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in csfbiomk1.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = csfbiomk1.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            csfbiomk1.loc[(csfbiomk1['checkin_bin'] == label) & (csfbiomk1[column].isnull()), column] = averages[label]

print(csfbiomk1.head())



   subject_id  visit  checkin_bin  exposurebin  age_decade  racecat_combined  \
0      1001.0      1            2            1         1.0               5.0   
1      1002.0      1            2            1         1.0               5.0   
2      1003.0      1            2            1         1.0               5.0   
3      1004.0      1            1            1         2.0               5.0   
4      1005.0      1            3            0         2.0               5.0   

   eduyears  totyr_foot  chiiseas_pf  chiiyrs_pf  ...    c_pT181  \
0      16.0         7.0       4335.4      2167.7  ...  24.400000   
1      15.0        14.0      10363.1      5708.1  ...  32.220000   
2      18.0        12.0       6685.4      4863.9  ...  32.220000   
3      16.0        16.0       7701.2      6448.9  ...  39.058000   
4      21.0         NaN          NaN         NaN  ...  38.311628   

   c_tTau_FLAG_below_ref_range  c_FLAG_hemolysis  c_tTau_FLAG_insufficient  \
0                     0.000000  

In [14]:
csfbiomk1 = csfbiomk1.drop(columns=csfbiomk1.loc[:,'subject_id':'chiiyrs_pg'].columns)
csfbiomk1 = csfbiomk1.drop(columns='c_strem2_FLAG_insufficient')

csfbiomk1 = csfbiomk1.drop(columns=['c_PDGFRbeta_FLAG_insufficient',
                              'c_tTau_FLAG_below_ref_range',
                              'c_FLAG_hemolysis', 
                              'c_tTau_FLAG_insufficient',
                              'dxcte_ptau217_csf_flag_dv'])
csfbiomk1.info()
csfbiomk1.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   c_strem2     236 non-null    float64
 1   c_PDGFRbeta  236 non-null    float64
 2   c_Ab40       236 non-null    float64
 3   c_Ab42       236 non-null    float64
 4   c_pT181      236 non-null    float64
 5   c_ttau       236 non-null    float64
 6   c_NfL        236 non-null    float64
 7   c_GFAP       236 non-null    float64
 8   c_pT231      236 non-null    float64
 9   c_pT217      236 non-null    float64
dtypes: float64(10)
memory usage: 18.6 KB


,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217
0,2035.000000,295.000000,8046.000000,702.000000,24.400000,598.000000,373.960000,8706.690000,487.340000,0.605900
1,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
2,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
3,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,2.080534
4,2752.976744,410.348837,11663.651163,964.255814,38.311628,393.948718,756.707209,8165.744286,525.304762,1.457107


# Preparing CSF Biomrarkers 2

In [15]:
# Check for null values in each column
csfbiomk2_nulls = csfbiomk2.isnull().sum()

csfbiomk2_nulls_df = pd.DataFrame({
    'Column': csfbiomk2_nulls.index,
    'Number of Nulls': csfbiomk2_nulls.values
})

csfbiomk2_nulls_df_transposed = csfbiomk2_nulls_df.T
csfbiomk2_nulls_df_sorted = csfbiomk2_nulls_df.sort_values(by='Number of Nulls', ascending=False)

print("Null values in csfbiomk2 (sorted):")
print(csfbiomk2_nulls_df_sorted)

# print("Null values in csfbiomk2:")
# print(csfbiomk2_nulls_df)
# csfbiomk2_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk2_nulls.csv", index=False)
csfbiomk2_nulls_df_sorted.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk2_nulls_sorted.csv", index=False)


Null values in csfbiomk2 (sorted):
                   Column  Number of Nulls
130                c_asyn               85
129             c_asyn_cv               85
131  dxcte_asyn_c_flag_dv               60
36                c_IL_21               60
27              c_IL_3_cv               60
..                    ...              ...
5        racecat_combined                0
4              age_decade                0
3             exposurebin                0
2             checkin_bin                0
0              subject_id                0

[132 rows x 2 columns]


In [16]:
# No need to remove unimportant columns

In [17]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in csfbiomk2.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = csfbiomk2.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            csfbiomk2.loc[(csfbiomk2['checkin_bin'] == label) & (csfbiomk2[column].isnull()), column] = averages[label]

print(csfbiomk2.head())

   subject_id  visit  checkin_bin  exposurebin  age_decade  racecat_combined  \
0      1001.0      1            2            1         1.0               5.0   
1      1002.0      1            2            1         1.0               5.0   
2      1003.0      1            2            1         1.0               5.0   
3      1004.0      1            1            1         2.0               5.0   
4      1005.0      1            3            0         2.0               5.0   

   eduyears  totyr_foot  chiiseas_pf  chiiyrs_pf  ...    c_IL_5  c_IL_7_cv  \
0      16.0         7.0       4335.4      2167.7  ...  0.789000   1.070000   
1      15.0        14.0      10363.1      5708.1  ...  1.173300   3.140625   
2      18.0        12.0       6685.4      4863.9  ...  1.173300   3.140625   
3      16.0        16.0       7701.2      6448.9  ...  1.138311   3.706485   
4      21.0         NaN          NaN         NaN  ...  1.167693   3.543143   

     c_IL_7  c_TNF_beta_cv  c_TNF_beta  c_VEGF_A_c

In [18]:
csfbiomk2 = csfbiomk2.drop(columns=csfbiomk2.loc[:,'subject_id':'chiiyrs_pg'].columns)
csfbiomk2 = csfbiomk2.drop(columns='dxcte_albumin_csf_flag_dv') # This is not for this dataset unless you find a column where there are too many nulls

csfbiomk2.head()

,Qalb,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,c_IL_17D,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,7.852013,30.721000,7.790000,0.089800,1.090000,16.200000,9.240000,0.047700,0.646000,384.000000,...,0.789000,1.070000,0.653000,2.400000,0.159000,2.220000,2.090000,2.895742,325.645161,1.000000
1,6.994813,24.907725,6.957075,0.178372,5.329725,12.402000,6.873625,0.040820,5.675050,394.600000,...,1.173300,3.140625,0.839475,4.056900,0.092031,2.473670,1.819000,2.895742,325.645161,0.225000
2,6.994813,24.907725,6.957075,0.178372,5.329725,12.402000,6.873625,0.040820,5.675050,394.600000,...,1.173300,3.140625,0.839475,4.056900,0.092031,2.473670,1.819000,2.895742,325.645161,0.225000
3,7.867197,26.628787,6.436894,0.189630,3.025862,14.650000,7.775840,0.042614,3.253543,393.734043,...,1.138311,3.706485,0.813897,4.826340,0.122339,3.713035,1.933536,2.418984,327.195122,0.127660
4,7.429948,24.686000,6.425048,0.195845,2.677810,13.692381,7.635476,0.046441,3.089405,403.214286,...,1.167693,3.543143,0.717286,5.541952,0.092165,4.012319,1.927667,2.547842,364.552632,0.095238


# Preparing CSF Biomrarkers 3

In [19]:
# Check for null values in each column
csfbiomk3_nulls = csfbiomk3.isnull().sum()

csfbiomk3_nulls_df = pd.DataFrame({
    'Column': csfbiomk3_nulls.index,
    'Number of Nulls': csfbiomk3_nulls.values
})

csfbiomk3_nulls_df_transposed = csfbiomk3_nulls_df.T
csfbiomk3_nulls_df_sorted = csfbiomk3_nulls_df.sort_values(by='Number of Nulls', ascending=False)

# print("Null values in csfbiomk3 (sorted):")
# print(csfbiomk3_nulls_df_sorted)

# print("Null values in csfbiomk3:")
# print(csfbiomk3_nulls_df)
# csfbiomk3_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk3_nulls.csv", index=False)
csfbiomk3_nulls_df_sorted.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk3_nulls_sorted.csv", index=False)


In [20]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in csfbiomk3.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = csfbiomk3.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            csfbiomk3.loc[(csfbiomk3['checkin_bin'] == label) & (csfbiomk3[column].isnull()), column] = averages[label]

print(csfbiomk3.head())


   subject_id  visit  checkin_bin  exposurebin  age_decade  racecat_combined  \
0      1001.0      1            2            1         1.0               5.0   
1      1002.0      1            2            1         1.0               5.0   
2      1003.0      1            2            1         1.0               5.0   
3      1004.0      1            1            1         2.0               5.0   
4      1005.0      1            3            0         2.0               5.0   

   eduyears  totyr_foot  chiiseas_pf  chiiyrs_pf  ...  chiiyrs_pl  \
0      16.0         7.0       4335.4      2167.7  ...     84942.5   
1      15.0        14.0      10363.1      5708.1  ...    118350.2   
2      18.0        12.0       6685.4      4863.9  ...    121198.9   
3      16.0        16.0       7701.2      6448.9  ...    159639.5   
4      21.0         NaN          NaN         NaN  ...         NaN   

   chiiseas_pg  chiiyrs_pg       c_dhpg        c_ne       c_dopa        c_da  \
0   14535346.6   7267673

In [21]:
csfbiomk3 = csfbiomk3.drop(columns=csfbiomk3.loc[:,'subject_id':'chiiyrs_pg'].columns)
csfbiomk3 = csfbiomk3.drop(columns=['c_catecholamine_flag_blood_dv','c_catecholamine_flag_diluted_dv']) # This is not for this dataset unless you find a column where there are too many nulls

csfbiomk3.head()

,c_dhpg,c_ne,c_dopa,c_da,c_dopac
0,1810.880000,201.770000,933.360000,0.100000,308.060000
1,2249.572857,195.530000,812.411714,13.893429,385.960000
2,2249.572857,195.530000,812.411714,13.893429,385.960000
3,2053.087059,156.803929,5134.040118,15.790941,322.747262
4,2173.238158,196.943684,937.913158,136.771579,401.465789


# Concatenating all Biomarkers

In [22]:
import pandas as pd
BloodCSF_biomk = pd.concat([bloodbiomk1, bloodbiomk2, csfbiomk1, csfbiomk2, csfbiomk3], axis=1)
# print(BloodCSF_biomk)


In [23]:
BloodCSF_biomk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Columns: 258 entries, p_Ab40 to c_dopac
dtypes: float64(258)
memory usage: 475.8 KB


# Now Calling Neuro-related Matrics

In [24]:
# calling Neuropsychiatric Measurements

briefa_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_20231117.csv"
briefa = pd.read_csv(briefa_file_path)
briefa = pd.DataFrame(briefa)


In [25]:
# Check for null values in each column
briefa_nulls = briefa.isnull().sum()

briefa_nulls_df = pd.DataFrame({
    'Column': briefa_nulls.index,
    'Number of Nulls': briefa_nulls.values
})

briefa_nulls_df_transposed = briefa_nulls_df.T

print("Null values in briefa:")
print(briefa_nulls_df)
briefa_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_nulls.csv", index=False)


Null values in briefa:
          Column  Number of Nulls
0     subject_id                0
1          visit                0
2    checkin_bin                0
3    exposurebin                0
4     age_decade                0
..           ...              ...
102      workorg                0
103    upseteasy                0
104    impulsive                0
105       pickup                0
106     complete                0

[107 rows x 2 columns]


In [26]:
briefa.head()

,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,activityorg,getover,onething,moodchange,consequence,workorg,upseteasy,impulsive,pickup,complete
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,2,1,2,1,1,2,2,1,1,2
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,3,3,3,2,3,3,3,3,2,3
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,3,3,3,3,2,3,3,3,1,2
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,1,1,1,1,1,1,2,1,1,1
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,1,2,1,1,1,1,1,1,1,1


In [27]:
briefa = briefa.drop(columns=briefa.loc[:,'subject_id':'chiiyrs_pg'].columns)
briefa = briefa.drop(columns=briefa.loc[:,'negativ':'complete'].columns)
briefa.head()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
0,75,64,114,57,39,46,12,9,11,7,14,17,20,13,11
1,101,85,171,85,70,79,17,15,23,15,19,24,27,15,16
2,90,77,158,79,68,77,15,14,27,12,20,22,22,14,12
3,43,40,81,42,38,46,9,8,15,6,9,8,11,7,8
4,43,39,81,41,38,45,9,11,12,6,10,8,10,7,8


In [28]:
briefa.describe()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
count,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000
mean,63.682203,56.724576,110.169492,56.508475,46.487288,55.203390,12.169492,9.690678,15.758475,8.868644,12.605932,13.758475,15.338983,9.466102,12.512712
std,19.879120,16.771696,33.473344,17.217382,14.657067,16.438698,3.606861,3.378543,5.769042,3.063115,4.101039,4.922909,5.199379,2.946855,4.335387
min,40.000000,36.000000,70.000000,35.000000,30.000000,36.000000,8.000000,6.000000,10.000000,6.000000,8.000000,8.000000,10.000000,6.000000,8.000000
25%,45.000000,41.000000,79.750000,41.000000,34.000000,41.000000,9.000000,6.000000,10.000000,6.000000,9.000000,9.000000,10.000000,6.000000,9.000000
50%,60.000000,53.000000,104.500000,54.000000,43.000000,51.000000,11.000000,9.000000,14.000000,8.000000,12.000000,13.000000,14.000000,9.000000,11.000000
75%,79.250000,68.250000,136.000000,69.000000,57.000000,67.250000,14.000000,12.000000,20.000000,11.000000,16.000000,18.000000,19.250000,12.000000,15.000000
max,116.000000,104.000000,204.000000,108.000000,88.000000,105.000000,23.000000,18.000000,30.000000,18.000000,23.000000,24.000000,29.000000,18.000000,24.000000


In [29]:
BloodCSF_biomk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Columns: 258 entries, p_Ab40 to c_dopac
dtypes: float64(258)
memory usage: 475.8 KB


In [30]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_BloodCSF_biomk'
folder_name = 'BloodCSF_biomk_to_briefa'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)


In [31]:
categories_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\categories.csv"
categories_df = pd.read_csv(categories_file_path)
new_column = categories_df['checkin_bin']
briefa['category'] = new_column
print(briefa.head())

BloodCSF_biomk['category'] = new_column
# print(bloodbiomk2.head())


    mi  tmi  gec  tgec  bri  tbri  inhibit  shift  emotcont  selfmon  \
0   75   64  114    57   39    46       12      9        11        7   
1  101   85  171    85   70    79       17     15        23       15   
2   90   77  158    79   68    77       15     14        27       12   
3   43   40   81    42   38    46        9      8        15        6   
4   43   39   81    41   38    45        9     11        12        6   

   initiate  workmem  planorg  taskmon  orgmat  category  
0        14       17       20       13      11         2  
1        19       24       27       15      16         2  
2        20       22       22       14      12         2  
3         9        8       11        7       8         1  
4        10        8       10        7       8         3  


In [32]:
# bloodbiomk1['category'] = briefa['category']

# correlation_matrix = bloodbiomk1.corr()

# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True)
# plt.title('Correlation Heatmap of Features')
# plt.show()

# # Alternatively, scatter plots for each feature vs y (checkin_bin)
# for feature in bloodbiomk1.columns[:-1]:  # Exclude the label column
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x='category', y=feature, data=bloodbiomk1)
#     plt.title(f'Boxplot of {feature} by category')
#     plt.xlabel('category')
#     plt.ylabel(feature)
#     plt.show()



# 95th percentile for both analysis and plotting

In [33]:
for feature in BloodCSF_biomk.columns[:-1]:  # Assuming 'category' is the last column
    upper_limit = BloodCSF_biomk[feature].quantile(0.95)  # Cap at 95th percentile
    lower_limit = BloodCSF_biomk[feature].quantile(0.05)  # Floor at 5th percentile
    BloodCSF_biomk[feature] = np.where(BloodCSF_biomk[feature] > upper_limit, upper_limit, BloodCSF_biomk[feature])
    BloodCSF_biomk[feature] = np.where(BloodCSF_biomk[feature] < lower_limit, lower_limit, BloodCSF_biomk[feature])

    

# Initial Step for Trainings

In [34]:
X = BloodCSF_biomk.copy()

In [35]:
X

,p_Ab40,p_Ab42,p_GFAP,p_NfL,p_PDGFRbeta,p_pT181,p_pT231,p_ttau,Qalb,s_alb,...,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv,c_dhpg,c_ne,c_dopa,c_da,c_dopac,category
0,126.5,8.535,73.80,11.950,5628.71,8.1200,7.5800,1.25000,7.852013,3.912500,...,2.090000,2.895742,325.645161,1.000000,1810.880000,201.770000,933.360000,1.625000,308.060000,2
1,84.2,7.460,42.05,6.440,10123.09,6.0400,7.5300,1.20000,6.994813,3.556088,...,1.819000,2.895742,325.645161,0.225000,2249.572857,195.530000,812.411714,13.893429,385.960000,2
2,124.0,7.285,34.35,8.220,10045.39,9.3300,9.3400,1.57000,6.994813,3.556088,...,1.819000,2.895742,325.645161,0.225000,2249.572857,195.530000,812.411714,13.893429,385.960000,2
3,110.5,5.920,48.45,13.000,9563.19,6.6300,5.8700,0.73000,7.867197,3.414635,...,1.933536,2.418984,327.195122,0.127660,2053.087059,156.803929,5134.040118,15.790941,322.747262,1
4,126.0,9.150,46.75,11.200,12826.15,8.3100,7.9900,1.54000,7.429948,3.378848,...,1.927667,2.547842,364.552632,0.095238,2173.238158,196.943684,937.913158,136.771579,401.465789,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,88.2,6.615,31.10,6.515,10722.07,22.1075,16.7175,2.98000,4.216818,3.367400,...,1.450000,4.070000,201.000000,0.000000,2340.500000,136.840000,694.030000,17.610000,548.290000,2
232,76.7,5.490,45.70,9.055,4444.06,4.7700,5.0200,2.37000,7.384902,3.279800,...,1.540000,2.070000,258.000000,0.000000,2612.910000,327.297500,966.820000,9.570000,548.290000,3
233,123.0,7.995,37.50,10.650,3908.37,4.0625,3.1225,1.66647,4.568572,3.771200,...,1.360000,4.110000,184.750000,0.000000,1264.492500,229.280000,748.040000,10.920000,181.790000,2
234,132.0,7.895,63.90,22.700,3582.92,11.2300,8.4600,1.08000,11.101832,2.926275,...,2.680000,1.150000,276.000000,0.000000,1851.100000,129.350000,659.440000,6.540000,202.350000,1


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# got error due to non numeric values, so removing them:

# Assume bloodbiomk1 is your DataFrame loaded with various types of data
X = BloodCSF_biomk.copy()

# Select only numeric columns from the DataFrame
X_numeric = X.select_dtypes(include=[np.number])
X = X_numeric

# Now X_numeric contains only the numeric columns of the original DataFrame
# Proceed with scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)


models = {}
predictions = {}


X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)

In [37]:
## Since I got error about duplicate columns, I am removing them here:
# Check for duplicate column names
duplicate_columns = X.columns[X.columns.duplicated()].tolist()
if duplicate_columns:
    print("Duplicate columns found:", duplicate_columns)

# Drop duplicate columns if desired
X = X.loc[:, ~X.columns.duplicated()]


Duplicate columns found: ['Qalb']


In [38]:
# Define the columns you want to select for modeling
## columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat'] 
selected_columns = ['mi']  # Replace 'column1' and 'column2' with the actual column names

# selected_columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat']  # Replace 'column1' and 'column2' with the actual column names


In [39]:
selected_columns

['mi']

In [40]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Initialize an empty list to store results
results = []
feature_importance_rankings = {}
model_name = "Random Forest"
selected_columns_str = "_".join(selected_columns)

# First, calculate feature importances for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Get feature importance and rank features
    feature_importance = pd.Series(model.feature_importances_, index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Save performance metrics with all features as a baseline
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")


# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to feature_importance_rankings.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")



Feature importance rankings saved to feature_importance_rankings.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [41]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Multiple Linear Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for linear regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [42]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression


# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gradient Boosting Machines"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LGBMRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11263
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 258
[LightGBM] [Info] Start training from score 63.660606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000053 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 694
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 17
[LightGBM] [Info] Start training from score 63.660606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1825
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 42
[LightGBM] [Info] Start training from score 63.660606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000700 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10477
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 239
[LightGBM] [Info] Start training from score 63.660606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000594 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10740
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 245
[LightGBM] [Info] Start training from score 63.660606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10995
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 251
[LightGBM] [Info] Start training from score 63.660606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11224
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 257
[LightGBM] [Info] Start training from score 63.660606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [43]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "XGBoost"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = xgb.XGBRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = xgb.XGBRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [44]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Support Vector Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for SVR
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train SVR model with all features as a baseline
    model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [45]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "ANN"
selected_columns_str = "_".join(selected_columns)

# Standardize features for ANN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train ANN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=X_train_selected.shape[1]),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_selected, y_train, epochs=100, batch_size=10, verbose=0)
    
    y_pred = model.predict(X_test_selected).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Scale selected features for ANN
        X_train_selected_scaled = scaler.fit_transform(X_train_selected)
        X_test_selected_scaled = scaler.transform(X_test_selected)
        
        # Train and evaluate ANN model with selected features
        model = Sequential([
            Dense(128, activation='relu', input_dim=X_train_selected_scaled.shape[1]),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train_selected_scaled, y_train, epochs=100, batch_size=10, verbose=0)
        
        y_pred = model.predict(X_test_selected_scaled).flatten()
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


3/3 [==============================] - 0s 811us/step


3/3 [==============================] - 0s 999us/step
Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [46]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "KNeighbors Regressor"
selected_columns_str = "_".join(selected_columns)

# Standardize features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train KNN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate KNN model with selected features
        model = KNeighborsRegressor(n_neighbors=5)
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [47]:
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Stacking Regressor"
selected_columns_str = "_".join(selected_columns)

# Define the base models and final estimator for stacking
base_models = [
    ('knn', KNeighborsRegressor(n_neighbors=13, leaf_size=23, p=1, weights='uniform')),
    ('svr', SVR(kernel='rbf', C=0.2, epsilon=0.01))
]
final_estimator = Ridge()
stack_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator, cv=5)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train stacking model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    stack_model.fit(X_train_selected, y_train)
    y_pred = stack_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate stacking model with selected features
        stack_model.fit(X_train_selected, y_train)
        y_pred = stack_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [48]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [49]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [50]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Lasso Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Lasso regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Lasso Regression Model with L1 regularization for feature selection
    model = Lasso(alpha=0.1)
    model.fit(X_train, y_train)
    
    # Select features based on non-zero coefficients
    selector = SelectFromModel(model, prefit=True)
    selected_features = X_train.columns[selector.get_support()]
    
    # Feature importance ranking based on absolute Lasso coefficients
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features

    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Lasso(alpha=0.1)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [51]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize lists and dictionaries to store results and feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Ridge Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Ridge regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Ridge Regression Model
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Ridge(alpha=1.0)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [52]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Initialize lists and dictionaries for storing results and feature rankings
results = []
feature_importance_rankings = {}
model_name = "Bayesian Ridge"
selected_columns_str = "_".join(selected_columns)

# Standardize features for consistency
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = BayesianRidge()
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance ranking
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = BayesianRidge()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [53]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gaussian Process Regression"
selected_columns_str = "_".join(selected_columns)

# Define the kernel
kernel = RBF(1.0)

# Standardize features for Gaussian Process Regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train the full Gaussian Process model as a baseline
    model = GaussianProcessRegressor(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_BloodCSF_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = GaussianProcessRegressor(kernel=kernel, random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_BloodCSF_biomk_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packag

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx
